In [354]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [355]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [356]:
baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'

In [357]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [358]:
response.json()

{'signingKey_hex': '6bb29b81fd186dda9837dcc0fd155ad942737a914ea75e47b76dd19bf83fc461',
 'traderId': '1',
 'verifyKey_hex': 'bac992093dc8dff24e49dd73ccf631bf75736f6c08ef14c171f498c394a2b0f2'}

In [359]:
response.json()

{'signingKey_hex': '6bb29b81fd186dda9837dcc0fd155ad942737a914ea75e47b76dd19bf83fc461',
 'traderId': '1',
 'verifyKey_hex': 'bac992093dc8dff24e49dd73ccf631bf75736f6c08ef14c171f498c394a2b0f2'}

In [360]:
url = baseurl + 'createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [361]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [362]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,1,1


In [363]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [364]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [365]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,1


In [366]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [367]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '1'}

In [368]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 10.232914924621582 seconds.
matched trade: 1 in 11.779691934585571 seconds.
matched trade: 2 in 9.516869068145752 seconds.
matched trade: 3 in 10.349142074584961 seconds.
matched trade: 4 in 14.344536066055298 seconds.
matched trade: 5 in 10.839059114456177 seconds.
matched trade: 6 in 11.26260495185852 seconds.
matched trade: 7 in 10.974135875701904 seconds.
matched trade: 8 in 9.916293144226074 seconds.
matched trade: 9 in 10.445212841033936 seconds.


In [ ]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

In [ ]:
response.json()

In [ ]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [ ]:
a = pd.read_json(response.json())
a

In [ ]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
a = pd.read_json(response.json())
a

In [ ]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


In [ ]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
pd.read_json(response.json())

In [ ]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
pd.read_json(response.json())

In [ ]:
tId